# Table of Contents

-  [About the dataset](#about)<br>
-  [Load the data](#load_data)<br>

# Imports

In [1]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline

In [29]:
# base modules
import os
import sys
import copy
import logging
from collections import OrderedDict

# for manipulating data
import numpy as np
import pandas as pd
import math
import dill

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance
from scipy.cluster import hierarchy

#for Time-Series Statistical Analysis
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt
import graphviz
import streamlit as st
import plotly.express as px
# seaborn
# altair


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings('ignore')

#display all columns
pd.options.display.max_columns = None

In [3]:
# path to repo
path_to_repo = os.getcwd()
path_to_repo

'/Users/nicolas/Desktop/Etudes/3_emlyon/2021:2022/Machine_Learning/project/notebook'

# Introduction: Crops Yield Project

### Project Ideas

Several ideas might pop up looking at this dataset, for example:

**Exploratory Data Analysis**
- Vizualisation of certain crops yields' temporal evolution per country
- Correlation between production & population

**Machine Learning**
- C02 emissions of crops using regression
- Clustering on CO2 crop emissioners
- Crop yields forecasting through time-series analysis using production & pop correlation

# Datasets Overview

## 1. Agricultural Crop Production

### Summary

Crop statistics for 173 products in Africa, America, Asia, Europe and Oceania, collected from 1961 to 2019.

### Description

Data from the Food and Agriculture Organization of the United Nations (FAO)
<br/><br/>Achieving food security for all is at the heart of FAO's efforts - ensuring that people have regular access to enough quality food to lead active and healthy lives. Our three main objectives are: the eradication of hunger, food insecurity and malnutrition; the eradication of poverty and the promotion of economic and social progress for all; and the sustainable management and use of natural resources, including land, water, air, climate and genetic resources, for the benefit of present and future generations.
<br/><br/>Primary crops, fibre crops. Crop statistics are recorded for 173 commodities, covering the following categories: Primary crops, Primary fibre crops, Cereals, Secondary cereals, Citrus, Fruit, Jute and related fibres, Oilcake equivalent, Primary oilseeds, Dry vegetables, Roots and tubers, Green fruits and vegetables and Melons. Data are expressed in terms of area harvested, quantity produced, yield and quantity of seed. The aim is to provide comprehensive coverage of production of all primary crops for all countries and regions of the world.<br/><br/>
Source : Organisation des Nations Unies pour l'alimentation et l'agriculture (FAO)

## 2. Gas emissions Statistics

### Summary

The FAOSTAT domain Emissions Totals disseminates information estimates of CH4, N2O and CO2 emissions/removals and their aggregates in CO2eq in units of kilotonnes (kt, or 106 kg). 

### Description

The FAOSTAT domain Emissions Totals summarizes the greenhouse gas (GHG) emissions disseminated in the FAOSTAT Climate Change Emissions domains, generated from agriculture and forest land. They consist of methane (CH4), nitrous oxide (N2O) and carbon dioxide (CO2) emissions from crop and livestock activities, forest management and include land use and land use change processes. Data are computed at Tier 1 of the IPCC Guidelines for National greenhouse gas (GHG) Inventories (IPCC, 1996; 1997; 2000; 2002; 2006; 2014). Estimates are available by country, with global coverage for the period 1961–2019 with projections for 2030 and 2050 for some categories of emissions or 1990–2019 for others. The database is updated annually.

## 3. Population per country

The FAOSTAT Population module contains time series data on population, by sex and urban/rural. The series consist of both estimates and projections for different periods as available from the original sources, namely:
1. Population data refers to the World Population Prospects: The 2019 Revision from the UN Population Division.
2. Urban/rural population data refers to the World Urbanization Prospects: The 2018 Revision from the UN Population Division.

# Data Loading

## 1. Load the *Agricultural Crop data*

### Get a glimpse of one of the main datasets

In [4]:
# Loading Africa's dataset

url = 'https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/Production_Crops_E_Africa.csv'
crops_africa = pd.read_csv(url, sep=',',encoding='latin-1')
crops_africa.head(10)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,Y1962F,Y1963,Y1963F,Y1964,Y1964F,Y1965,Y1965F,Y1966,Y1966F,Y1967,Y1967F,Y1968,Y1968F,Y1969,Y1969F,Y1970,Y1970F,Y1971,Y1971F,Y1972,Y1972F,Y1973,Y1973F,Y1974,Y1974F,Y1975,Y1975F,Y1976,Y1976F,Y1977,Y1977F,Y1978,Y1978F,Y1979,Y1979F,Y1980,Y1980F,Y1981,Y1981F,Y1982,Y1982F,Y1983,Y1983F,Y1984,Y1984F,Y1985,Y1985F,Y1986,Y1986F,Y1987,Y1987F,Y1988,Y1988F,Y1989,Y1989F,Y1990,Y1990F,Y1991,Y1991F,Y1992,Y1992F,Y1993,Y1993F,Y1994,Y1994F,Y1995,Y1995F,Y1996,Y1996F,Y1997,Y1997F,Y1998,Y1998F,Y1999,Y1999F,Y2000,Y2000F,Y2001,Y2001F,Y2002,Y2002F,Y2003,Y2003F,Y2004,Y2004F,Y2005,Y2005F,Y2006,Y2006F,Y2007,Y2007F,Y2008,Y2008F,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F
0,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,F,13300.0,F,13300.0,F,14200.0,F,13800.0,F,12700.0,F,12900.0,F,10000.0,F,5500.0,F,6700.0,F,4600.0,F,8300.0,F,9200.0,F,10150.0,F,11000.0,F,18250.0,F,10400.0,F,7100.0,F,8100.0,F,8700.0,F,15960.0,NaN,15450.0,NaN,18030.0,NaN,16490.0,NaN,21100.0,NaN,20860.0,NaN,21000.0,NaN,21370.0,NaN,23610.0,NaN,25010.0,NaN,24240.0,NaN,24970.0,NaN,25880.0,NaN,26130.0,NaN,24860.0,NaN,25190.0,NaN,27440.0,NaN,26490.0,NaN,26820.0,NaN,27150.0,NaN,26980.0,NaN,27720.0,NaN,30630.0,NaN,30453.0,NaN,35099.0,NaN,39098.0,NaN,40890.0,NaN,39787.0,NaN,39313.0,NaN,54485.0,NaN,52245.0,NaN,49975.0,NaN,49011.0,NaN,39050.0,NaN,40403.0,NaN,49983.0,NaN,50100.0,NaN,43043.0,NaN,35380.0,NaN
1,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,Fc,4511.0,Fc,4511.0,Fc,4507.0,Fc,4493.0,Fc,4488.0,Fc,4510.0,Fc,4497.0,Fc,4484.0,Fc,4478.0,Fc,4500.0,Fc,4514.0,Fc,4502.0,Fc,4500.0,Fc,4542.0,Fc,4498.0,Fc,4487.0,Fc,4496.0,Fc,4483.0,Fc,4483.0,Fc,4969.0,Fc,4296.0,Fc,3934.0,Fc,4534.0,Fc,4716.0,Fc,3864.0,Fc,3930.0,Fc,4528.0,Fc,5268.0,Fc,4691.0,Fc,6505.0,Fc,7169.0,Fc,9538.0,Fc,7312.0,Fc,7992.0,Fc,13323.0,Fc,7069.0,Fc,8169.0,Fc,9546.0,Fc,9754.0,Fc,9340.0,Fc,11648.0,Fc,10524.0,Fc,12473.0,Fc,12929.0,Fc,13728.0,Fc,8342.0,Fc,9933.0,Fc,12055.0,Fc,10457.0,Fc,9689.0,Fc,13304.0,Fc,12965.0,Fc,16601.0,Fc,18930.0,Fc,13223.0,Fc,12362.0,Fc,13292.0,Fc,20467.0,Fc
2,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,NaN,6000.0,NaN,6000.0,NaN,6400.0,NaN,6200.0,NaN,5700.0,NaN,5818.0,NaN,4497.0,NaN,2466.0,NaN,3000.0,NaN,2070.0,NaN,3747.0,NaN,4142.0,NaN,4567.0,NaN,4996.0,NaN,8208.0,NaN,4666.0,NaN,3192.0,NaN,3631.0,NaN,3900.0,NaN,7930.0,NaN,6637.0,NaN,7093.0,NaN,7476.0,NaN,9951.0,NaN,8060.0,NaN,8253.0,NaN,9676.0,NaN,12438.0,NaN,11733.0,NaN,15768.0,NaN,17901.0,NaN,24685.0,NaN,19106.0,NaN,19869.0,NaN,33561.0,NaN,19396.0,NaN,21641.0,NaN,25602.0,NaN,26483.0,NaN,25199.0,NaN,32287.0,NaN,32234.0,NaN,37985.0,NaN,45379.0,NaN,53673.0,NaN,34110.0,NaN,39521.0,NaN,47393.0,NaN,56973.0,NaN,50621.0,NaN,66487.0,NaN,63545.0,NaN,64827.0,NaN,76482.0,NaN,66095.0,NaN,61934.0,NaN,57213.0,NaN,72412.0,NaN
3,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,F,3100.0,F,2800.0,F,2700.0,F,2900.0,F,2500.0,F,2500.0,F,2300.0,F,2600.0,F,2800.0,F,3000.0,F,2500.0,F,2200.0,F,3200.0,F,4300.0,F,3200.0,F,3800.0,F,3200.0,F,3000.0,F,4300.0,F,4600.0,F,4900.0,F,4800.0,F,7800.0,F,8000.0,F,8000.0,F,8700.0,F,8000.0,F,10200.0,F,10740.0,NaN,10960.0,NaN,11140.0,NaN,12020.0,NaN,11800.0,NaN,11930.0,NaN,12190.0,NaN,12260.0,NaN,12870.0,NaN,13020.0,NaN,13480.0,NaN,14040.0,NaN,15240.0,NaN,18080.0,NaN,19861.0,NaN,24279.0,NaN,28658.0,NaN,31904.0,NaN,33206.0,NaN,36616.0,NaN,52419.0,NaN,51080.0,NaN,48828.0,NaN,48064.0,NaN,40418.0,NaN,41011.0,NaN,46070.0,NaN,44620.0,NaN,39034.0,NaN,32989.0,NaN
4,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,Fc,45161.0,Fc,46429.0,Fc,46078.0,Fc,45348.0,Fc,44832.0,Fc,45456.0,Fc,45609.0,Fc,45681.0,Fc,46071.0,Fc,45020.0,Fc,45920.0,Fc,45736.0,Fc,45525.0,Fc,50174.0,Fc,45700.0,Fc,50471.0,Fc,48678.0,Fc,45520.0,Fc,50740.0,Fc,50839.0,Fc,49951.0,Fc,49825.0,Fc,49810.0,Fc,53000.0,Fc,50874.0,Fc,50216.0,Fc,49975.0,Fc,50387.0,Fc,36258.0,Fc,43638.0,Fc,5564

In [5]:
crops_africa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Columns: 125 entries, Area Code to Y2019F
dtypes: float64(59), int64(3), object(63)
memory usage: 8.7+ MB


### Load all datasets

In [6]:
def open_all_datasets_df(item):
    df = pd.read_csv('https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/Production_Crops_E_' + str(item) + '.csv', low_memory = False, encoding='latin1')
    df['Continent'] = item
    return df

In [7]:
continents = ['Africa','Americas','Asia','Europe','Oceania']

In [8]:
crops_raw = pd.DataFrame()

for continent in continents:
    crops_raw = crops_raw.append(open_all_datasets_df(continent))

In [9]:
crops_raw

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,Y1962F,Y1963,Y1963F,Y1964,Y1964F,Y1965,Y1965F,Y1966,Y1966F,Y1967,Y1967F,Y1968,Y1968F,Y1969,Y1969F,Y1970,Y1970F,Y1971,Y1971F,Y1972,Y1972F,Y1973,Y1973F,Y1974,Y1974F,Y1975,Y1975F,Y1976,Y1976F,Y1977,Y1977F,Y1978,Y1978F,Y1979,Y1979F,Y1980,Y1980F,Y1981,Y1981F,Y1982,Y1982F,Y1983,Y1983F,Y1984,Y1984F,Y1985,Y1985F,Y1986,Y1986F,Y1987,Y1987F,Y1988,Y1988F,Y1989,Y1989F,Y1990,Y1990F,Y1991,Y1991F,Y1992,Y1992F,Y1993,Y1993F,Y1994,Y1994F,Y1995,Y1995F,Y1996,Y1996F,Y1997,Y1997F,Y1998,Y1998F,Y1999,Y1999F,Y2000,Y2000F,Y2001,Y2001F,Y2002,Y2002F,Y2003,Y2003F,Y2004,Y2004F,Y2005,Y2005F,Y2006,Y2006F,Y2007,Y2007F,Y2008,Y2008F,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Continent
0,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,F,13300.0,F,13300.0,F,14200.0,F,13800.0,F,12700.0,F,12900.0,F,10000.0,F,5500.0,F,6700.0,F,4600.0,F,8300.0,F,9200.0,F,10150.0,F,11000.0,F,18250.0,F,10400.0,F,7100.0,F,8100.0,F,8700.0,F,15960.0,NaN,15450.0,NaN,18030.0,NaN,16490.0,NaN,21100.0,NaN,20860.0,NaN,21000.0,NaN,21370.0,NaN,23610.0,NaN,25010.0,NaN,24240.0,NaN,24970.0,NaN,25880.0,NaN,26130.0,NaN,24860.0,NaN,25190.0,NaN,27440.0,NaN,26490.0,NaN,26820.0,NaN,27150.0,NaN,26980.0,NaN,27720.0,NaN,30630.0,NaN,30453.0,NaN,35099.0,NaN,39098.0,NaN,40890.0,NaN,39787.0,NaN,39313.0,NaN,54485.0,NaN,52245.0,NaN,49975.0,NaN,49011.0,NaN,39050.0,NaN,40403.0,NaN,49983.0,NaN,50100.0,NaN,43043.0,NaN,35380.0,NaN,Africa
1,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,Fc,4511.0,Fc,4511.0,Fc,4507.0,Fc,4493.0,Fc,4488.0,Fc,4510.0,Fc,4497.0,Fc,4484.0,Fc,4478.0,Fc,4500.0,Fc,4514.0,Fc,4502.0,Fc,4500.0,Fc,4542.0,Fc,4498.0,Fc,4487.0,Fc,4496.0,Fc,4483.0,Fc,4483.0,Fc,4969.0,Fc,4296.0,Fc,3934.0,Fc,4534.0,Fc,4716.0,Fc,3864.0,Fc,3930.0,Fc,4528.0,Fc,5268.0,Fc,4691.0,Fc,6505.0,Fc,7169.0,Fc,9538.0,Fc,7312.0,Fc,7992.0,Fc,13323.0,Fc,7069.0,Fc,8169.0,Fc,9546.0,Fc,9754.0,Fc,9340.0,Fc,11648.0,Fc,10524.0,Fc,12473.0,Fc,12929.0,Fc,13728.0,Fc,8342.0,Fc,9933.0,Fc,12055.0,Fc,10457.0,Fc,9689.0,Fc,13304.0,Fc,12965.0,Fc,16601.0,Fc,18930.0,Fc,13223.0,Fc,12362.0,Fc,13292.0,Fc,20467.0,Fc,Africa
2,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,NaN,6000.0,NaN,6000.0,NaN,6400.0,NaN,6200.0,NaN,5700.0,NaN,5818.0,NaN,4497.0,NaN,2466.0,NaN,3000.0,NaN,2070.0,NaN,3747.0,NaN,4142.0,NaN,4567.0,NaN,4996.0,NaN,8208.0,NaN,4666.0,NaN,3192.0,NaN,3631.0,NaN,3900.0,NaN,7930.0,NaN,6637.0,NaN,7093.0,NaN,7476.0,NaN,9951.0,NaN,8060.0,NaN,8253.0,NaN,9676.0,NaN,12438.0,NaN,11733.0,NaN,15768.0,NaN,17901.0,NaN,24685.0,NaN,19106.0,NaN,19869.0,NaN,33561.0,NaN,19396.0,NaN,21641.0,NaN,25602.0,NaN,26483.0,NaN,25199.0,NaN,32287.0,NaN,32234.0,NaN,37985.0,NaN,45379.0,NaN,53673.0,NaN,34110.0,NaN,39521.0,NaN,47393.0,NaN,56973.0,NaN,50621.0,NaN,66487.0,NaN,63545.0,NaN,64827.0,NaN,76482.0,NaN,66095.0,NaN,61934.0,NaN,57213.0,NaN,72412.0,NaN,Africa
3,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,F,3100.0,F,2800.0,F,2700.0,F,2900.0,F,2500.0,F,2500.0,F,2300.0,F,2600.0,F,2800.0,F,3000.0,F,2500.0,F,2200.0,F,3200.0,F,4300.0,F,3200.0,F,3800.0,F,3200.0,F,3000.0,F,4300.0,F,4600.0,F,4900.0,F,4800.0,F,7800.0,F,8000.0,F,8000.0,F,8700.0,F,8000.0,F,10200.0,F,10740.0,NaN,10960.0,NaN,11140.0,NaN,12020.0,NaN,11800.0,NaN,11930.0,NaN,12190.0,NaN,12260.0,NaN,12870.0,NaN,13020.0,NaN,13480.0,NaN,14040.0,NaN,15240.0,NaN,18080.0,NaN,19861.0,NaN,24279.0,NaN,28658.0,NaN,31904.0,NaN,33206.0,NaN,36616.0,NaN,52419.0,NaN,51080.0,NaN,48828.0,NaN,48064.0,NaN,40418.0,NaN,41011.0,NaN,46070.0,NaN,44620.0,NaN,39034.0,NaN,32989.0,NaN,Africa
4,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,Fc,45161.0,Fc,46429.0,Fc,46078.0,Fc,45348.0,Fc,44832.0,Fc,45456.0,Fc,45609.0,Fc,45681.0,Fc,46071.0,Fc,45020.0,Fc,45920.0,Fc,45736.0,Fc,45525.0,Fc,50174.0,Fc,45700.0,Fc,50471.0,Fc,48678.0,Fc,45520.0,Fc,50740.0,Fc,50839.0,Fc,49951.0,Fc,49825.0,Fc,49810.0,Fc,53000.0,Fc,50874.0,Fc,50216.0,Fc,49975.0,Fc

It appears that columns representing the data per year come with an indicator "F", let's dig into it !

In [10]:
flags_df = pd.read_csv("https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/flags.csv", encoding="latin1")
flags_df

,"ï»¿""Flag""",Flags
0,NaN,Official data
1,*,Unofficial figure
2,A,"Aggregate, may include official, semi-official..."
3,Bk,Break in series
4,C,Calculated
5,Ce,Calculated data based on estimated data
6,Cv,Calculated through value
7,E,Expert sources from FAO (including other divis...
8,F,FAO estimate
9,Fb,Data obtained as a balance


Each year column comes with another column, stating the source of the figures for this year (whether the figure was calculated with official data, estimated, etc).<br/>
It is good info to know, but let's put these data pieces aside and keep a smaller df:

In [11]:
crops_all = copy.deepcopy(crops_raw)
crops_all = crops_all.loc[:, ~crops_all.columns.str.endswith('F')]
crops_all = crops_all[crops_all.columns[~crops_all.columns.str.endswith('F')]]
first_col = crops_all.pop('Continent')
crops_all.insert(0, 'Continent', first_col)
crops_all

,Continent,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,Africa,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,13300.0,13300.0,14200.0,13800.0,12700.0,12900.0,10000.0,5500.0,6700.0,4600.0,8300.0,9200.0,10150.0,11000.0,18250.0,10400.0,7100.0,8100.0,8700.0,15960.0,15450.0,18030.0,16490.0,21100.0,20860.0,21000.0,21370.0,23610.0,25010.0,24240.0,24970.0,25880.0,26130.0,24860.0,25190.0,27440.0,26490.0,26820.0,27150.0,26980.0,27720.0,30630.0,30453.0,35099.0,39098.0,40890.0,39787.0,39313.0,54485.0,52245.0,49975.0,49011.0,39050.0,40403.0,49983.0,50100.0,43043.0,35380.0
1,Africa,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,4511.0,4511.0,4507.0,4493.0,4488.0,4510.0,4497.0,4484.0,4478.0,4500.0,4514.0,4502.0,4500.0,4542.0,4498.0,4487.0,4496.0,4483.0,4483.0,4969.0,4296.0,3934.0,4534.0,4716.0,3864.0,3930.0,4528.0,5268.0,4691.0,6505.0,7169.0,9538.0,7312.0,7992.0,13323.0,7069.0,8169.0,9546.0,9754.0,9340.0,11648.0,10524.0,12473.0,12929.0,13728.0,8342.0,9933.0,12055.0,10457.0,9689.0,13304.0,12965.0,16601.0,18930.0,13223.0,12362.0,13292.0,20467.0
2,Africa,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,6000.0,6000.0,6400.0,6200.0,5700.0,5818.0,4497.0,2466.0,3000.0,2070.0,3747.0,4142.0,4567.0,4996.0,8208.0,4666.0,3192.0,3631.0,3900.0,7930.0,6637.0,7093.0,7476.0,9951.0,8060.0,8253.0,9676.0,12438.0,11733.0,15768.0,17901.0,24685.0,19106.0,19869.0,33561.0,19396.0,21641.0,25602.0,26483.0,25199.0,32287.0,32234.0,37985.0,45379.0,53673.0,34110.0,39521.0,47393.0,56973.0,50621.0,66487.0,63545.0,64827.0,76482.0,66095.0,61934.0,57213.0,72412.0
3,Africa,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,3100.0,2800.0,2700.0,2900.0,2500.0,2500.0,2300.0,2600.0,2800.0,3000.0,2500.0,2200.0,3200.0,4300.0,3200.0,3800.0,3200.0,3000.0,4300.0,4600.0,4900.0,4800.0,7800.0,8000.0,8000.0,8700.0,8000.0,10200.0,10740.0,10960.0,11140.0,12020.0,11800.0,11930.0,12190.0,12260.0,12870.0,13020.0,13480.0,14040.0,15240.0,18080.0,19861.0,24279.0,28658.0,31904.0,33206.0,36616.0,52419.0,51080.0,48828.0,48064.0,40418.0,41011.0,46070.0,44620.0,39034.0,32989.0
4,Africa,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,45161.0,46429.0,46078.0,45348.0,44832.0,45456.0,45609.0,45681.0,46071.0,45020.0,45920.0,45736.0,45525.0,50174.0,45700.0,50471.0,48678.0,45520.0,50740.0,50839.0,49951.0,49825.0,49810.0,53000.0,50874.0,50216.0,49975.0,50387.0,36258.0,43638.0,55641.0,52753.0,41777.0,53764.0,60598.0,53446.0,58574.0,67065.0,71600.0,74715.0,79421.0,74968.0,83265.0,82257.0,98835.0,59557.0,78590.0,73047.0,72233.0,79112.0,81414.0,94860.0,114507.0,110086.0,108716.0,110766.0,124970.0,169399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,Oceania,155,Vanuatu,1720,"Roots and Tubers, Total",5419,Yield,hg/ha,225455.0,227273.0,229091.0,220870.0,216949.0,216667.0,208800.0,209375.0,207692.0,202222.0,196429.0,198571.0,195862.0,196552.0,195270.0,193333.0,200000.0,200000.0,200000.0,200000.0,188889.0,194737.0,194872.0,192213.0,130000.0,111429.0,86667.0,80851.0,80851.0,80412.0,76000.0,80000.0,79343.0,79326.0,79404.0,79484.0,79167.0,79614.0,79773.0,79592.0,81000.0,81161.0,81418.0,82692.0,81481.0,81818.0,81919.0,82727.0,82883.0,82072.0,80645.0,82258.0,82540.0,82518.0,82467.0,82306.0,82341.0,82374.0,82407.0
1650,Oceania,155,Vanuatu,1720,"Roots and Tubers, Total",5510,Production,tonnes,24800.0,25000.0,25200.0,25400.0,25600.0,26000.0,26100.0,26800.0,27000.0,27300.0,27500.0,27800.0,2840

## 2. Loading the *Gas Emissions* dataset

In [12]:
# Loading Gas Emissions' dataset

url = 'https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/emissions/emissions_full.csv'
emissions_df = pd.read_csv(url, sep=',',encoding='latin-1',low_memory=False)
emissions_df

,Code zone,Zone,Code Produit,Produit,Code Élément,Élément,Code source,Source,Unité,Y1961,Y1961F,Y1961N,Y1962,Y1962F,Y1962N,Y1963,Y1963F,Y1963N,Y1964,Y1964F,Y1964N,Y1965,Y1965F,Y1965N,Y1966,Y1966F,Y1966N,Y1967,Y1967F,Y1967N,Y1968,Y1968F,Y1968N,Y1969,Y1969F,Y1969N,Y1970,Y1970F,Y1970N,Y1971,Y1971F,Y1971N,Y1972,Y1972F,Y1972N,Y1973,Y1973F,Y1973N,Y1974,Y1974F,Y1974N,Y1975,Y1975F,Y1975N,Y1976,Y1976F,Y1976N,Y1977,Y1977F,Y1977N,Y1978,Y1978F,Y1978N,Y1979,Y1979F,Y1979N,Y1980,Y1980F,Y1980N,Y1981,Y1981F,Y1981N,Y1982,Y1982F,Y1982N,Y1983,Y1983F,Y1983N,Y1984,Y1984F,Y1984N,Y1985,Y1985F,Y1985N,Y1986,Y1986F,Y1986N,Y1987,Y1987F,Y1987N,Y1988,Y1988F,Y1988N,Y1989,Y1989F,Y1989N,Y1990,Y1990F,Y1990N,Y1991,Y1991F,Y1991N,Y1992,Y1992F,Y1992N,Y1993,Y1993F,Y1993N,Y1994,Y1994F,Y1994N,Y1995,Y1995F,Y1995N,Y1996,Y1996F,Y1996N,Y1997,Y1997F,Y1997N,Y1998,Y1998F,Y1998N,Y1999,Y1999F,Y1999N,Y2000,Y2000F,Y2000N,Y2001,Y2001F,Y2001N,Y2002,Y2002F,Y2002N,Y2003,Y2003F,Y2003N,Y2004,Y2004F,Y2004N,Y2005,Y2005F,Y2005N,Y2006,Y2006F,Y2006N,Y2007,Y2007F,Y2007N,Y2008,Y2008F,Y2008N,Y2009,Y2009F,Y2009N,Y2010,Y2010F,Y2010N,Y2011,Y2011F,Y2011N,Y2012,Y2012F,Y2012N,Y2013,Y2013F,Y2013N,Y2014,Y2014F,Y2014N,Y2015,Y2015F,Y2015N,Y2016,Y2016F,Y2016N,Y2017,Y2017F,Y2017N,Y2018,Y2018F,Y2018N,Y2019,Y2019F,Y2019N,Y2020,Y2020F,Y2020N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N
0,2,Afghanistan,5058,Fermentation entérique,7225,Émissions (CH4),3050,FAO TIER 1,kilotonnes,240.6831,Fc,NaN,245.3106,Fc,NaN,255.8285,Fc,NaN,259.065,Fc,NaN,265.598,Fc,NaN,276.994,Fc,NaN,280.094,Fc,NaN,288.8210,Fc,NaN,286.382,Fc,NaN,290.26,Fc,NaN,287.79,Fc,NaN,231.527,Fc,NaN,244.979,Fc,NaN,262.836,Fc,NaN,282.0740,Fc,NaN,288.225,Fc,NaN,280.876,Fc,NaN,280.2850,Fc,NaN,274.235,Fc,NaN,275.37,Fc,NaN,278.21,Fc,NaN,277.94,Fc,NaN,262.63,Fc,NaN,230.15,Fc,NaN,202.73,Fc,NaN,160.264,Fc,NaN,172.94,Fc,NaN,181.436,Fc,NaN,179.562,Fc,NaN,178.4682,Fc,NaN,187.5500,Fc,NaN,189.7600,Fc,NaN,190.8300,Fc,NaN,197.9210,Fc,NaN,211.1555,Fc,NaN,239.7213,Fc,NaN,264.6095,Fc,NaN,283.4502,Fc,NaN,318.3395,Fc,NaN,272.1400,Fc,NaN,225.4030,Fc,NaN,287.8930,Fc,NaN,293.6260,Fc,NaN,285.5980,Fc,NaN,295.4010,Fc,NaN,300.8070,Fc,NaN,304.2250,Fc,NaN,339.5590,Fc,NaN,345.6580,Fc,NaN,401.0680,Fc,NaN,402.5130,Fc,NaN,396.9210,Fc,NaN,393.0930,Fc,NaN,398.2870,Fc,NaN,383.3023,Fc,NaN,380.8767,Fc,NaN,371.2863,Fc,NaN,378.8870,Fc,NaN,389.6563,Fc,NaN,NaN,NaN,NaN,453.7474,Fc,NaN,603.6185,Fc,NaN
1,2,Afghanistan,5058,Fermentation entérique,7225,Émissions (CH4),3051,UNFCCC,kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,5058,Fermentation entérique,724413,Émissions (CO2eq) venant de CH4 (AR5),3050,FAO TIER 1,kilotonnes,6739.1279,Fc,NaN,6868.6971,Fc,NaN,7163.1986,Fc,NaN,7253.820,Fc,NaN,7436.744,Fc,NaN,7755.832,Fc,NaN,7842.632,Fc,NaN,8086.9891,Fc,NaN,8018.696,Fc,NaN,8127.28,Fc,NaN,8058.12,Fc,NaN,6482.756,Fc,NaN,6859.412,Fc,NaN,7359.408,Fc,NaN,7898.0731,Fc,NaN,8070.300,Fc,NaN,7864.528,Fc,NaN,7847.9811,Fc,NaN,7678.580,Fc,NaN,7710.36,Fc,NaN,7789.88,Fc,NaN,7782.32,Fc,NaN,7353.64,Fc,NaN,6444.20,Fc,NaN,5676.44,Fc,NaN,4487.392,Fc,NaN,4842.32,Fc,NaN,5080.208,Fc,NaN,5027.736,Fc,NaN,4997.1108,Fc,NaN,5251.4000,Fc,NaN,5313.2800,Fc,NaN,5343.2400,Fc,NaN,5541.7880,Fc,NaN,5912.3546,Fc,NaN,6712.1964,Fc,NaN,7409.0660,Fc,NaN,7936.6067,Fc,NaN,8913.5049,Fc,NaN,7619.9200,Fc,NaN,6311.2840,Fc,NaN,8061.0

**Let's focus for now on CO2 emissions**

In [13]:
co2_df = emissions_df.drop(emissions_df[emissions_df['Code Élément'] != 7273].index)
co2_df

,Code zone,Zone,Code Produit,Produit,Code Élément,Élément,Code source,Source,Unité,Y1961,Y1961F,Y1961N,Y1962,Y1962F,Y1962N,Y1963,Y1963F,Y1963N,Y1964,Y1964F,Y1964N,Y1965,Y1965F,Y1965N,Y1966,Y1966F,Y1966N,Y1967,Y1967F,Y1967N,Y1968,Y1968F,Y1968N,Y1969,Y1969F,Y1969N,Y1970,Y1970F,Y1970N,Y1971,Y1971F,Y1971N,Y1972,Y1972F,Y1972N,Y1973,Y1973F,Y1973N,Y1974,Y1974F,Y1974N,Y1975,Y1975F,Y1975N,Y1976,Y1976F,Y1976N,Y1977,Y1977F,Y1977N,Y1978,Y1978F,Y1978N,Y1979,Y1979F,Y1979N,Y1980,Y1980F,Y1980N,Y1981,Y1981F,Y1981N,Y1982,Y1982F,Y1982N,Y1983,Y1983F,Y1983N,Y1984,Y1984F,Y1984N,Y1985,Y1985F,Y1985N,Y1986,Y1986F,Y1986N,Y1987,Y1987F,Y1987N,Y1988,Y1988F,Y1988N,Y1989,Y1989F,Y1989N,Y1990,Y1990F,Y1990N,Y1991,Y1991F,Y1991N,Y1992,Y1992F,Y1992N,Y1993,Y1993F,Y1993N,Y1994,Y1994F,Y1994N,Y1995,Y1995F,Y1995N,Y1996,Y1996F,Y1996N,Y1997,Y1997F,Y1997N,Y1998,Y1998F,Y1998N,Y1999,Y1999F,Y1999N,Y2000,Y2000F,Y2000N,Y2001,Y2001F,Y2001N,Y2002,Y2002F,Y2002N,Y2003,Y2003F,Y2003N,Y2004,Y2004F,Y2004N,Y2005,Y2005F,Y2005N,Y2006,Y2006F,Y2006N,Y2007,Y2007F,Y2007N,Y2008,Y2008F,Y2008N,Y2009,Y2009F,Y2009N,Y2010,Y2010F,Y2010N,Y2011,Y2011F,Y2011N,Y2012,Y2012F,Y2012N,Y2013,Y2013F,Y2013N,Y2014,Y2014F,Y2014N,Y2015,Y2015F,Y2015N,Y2016,Y2016F,Y2016N,Y2017,Y2017F,Y2017N,Y2018,Y2018F,Y2018N,Y2019,Y2019F,Y2019N,Y2020,Y2020F,Y2020N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N
47,2,Afghanistan,6750,Conversion nette de forêt,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2,Afghanistan,6751,Terres forestières,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,-2388.8030,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,121.9016,Fc,NaN,-246.2191,Fc,NaN,-246.2191,Fc,NaN,-246.2191,Fc,NaN,-246.2191,Fc,NaN,-246.2191,Fc,NaN,154.6574,Fc,NaN,154.6574,Fc,NaN,154.6574,Fc,NaN,154.6574,Fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2,Afghanistan,6993,Feux de tourbières,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,NaN,0.0000,Fc,

## 3. Loading the *Population Stats* dataset

In [14]:
#Loading the population_stats dataset
pop_df_raw = pd.read_csv('https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/pop_data_onu.csv',thousands=' ')
pop_df_raw

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486
...,...,...,...,...,...,...,...,...,...,...
280927,716,Zimbabwe,207,Lower 95 PI,2080,2080.5,10576.533,11255.983,21836.893,56.448
280928,716,Zimbabwe,207,Lower 95 PI,2085,2085.5,10293.349,11050.875,21355.988,55.205
280929,716,Zimbabwe,207,Lower 95 PI,2090,2090.5,9920.336,10767.709,20689.956,53.483
280930,716,Zimbabwe,207,Lower 95 PI,2095,2095.5,9503.711,10412.184,19892.080,51.421


# Exploratory Data Analysis

Before trying to preprocess the data in order to make use of it for our machine learning tasks, let's explore what we have & try to figure out some primary findings.

## 1. Exploring the Agricultural Crop data

In [15]:
crops_all.head()

,Continent,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,Africa,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,13300.0,13300.0,14200.0,13800.0,12700.0,12900.0,10000.0,5500.0,6700.0,4600.0,8300.0,9200.0,10150.0,11000.0,18250.0,10400.0,7100.0,8100.0,8700.0,15960.0,15450.0,18030.0,16490.0,21100.0,20860.0,21000.0,21370.0,23610.0,25010.0,24240.0,24970.0,25880.0,26130.0,24860.0,25190.0,27440.0,26490.0,26820.0,27150.0,26980.0,27720.0,30630.0,30453.0,35099.0,39098.0,40890.0,39787.0,39313.0,54485.0,52245.0,49975.0,49011.0,39050.0,40403.0,49983.0,50100.0,43043.0,35380.0
1,Africa,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,4511.0,4511.0,4507.0,4493.0,4488.0,4510.0,4497.0,4484.0,4478.0,4500.0,4514.0,4502.0,4500.0,4542.0,4498.0,4487.0,4496.0,4483.0,4483.0,4969.0,4296.0,3934.0,4534.0,4716.0,3864.0,3930.0,4528.0,5268.0,4691.0,6505.0,7169.0,9538.0,7312.0,7992.0,13323.0,7069.0,8169.0,9546.0,9754.0,9340.0,11648.0,10524.0,12473.0,12929.0,13728.0,8342.0,9933.0,12055.0,10457.0,9689.0,13304.0,12965.0,16601.0,18930.0,13223.0,12362.0,13292.0,20467.0
2,Africa,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,6000.0,6000.0,6400.0,6200.0,5700.0,5818.0,4497.0,2466.0,3000.0,2070.0,3747.0,4142.0,4567.0,4996.0,8208.0,4666.0,3192.0,3631.0,3900.0,7930.0,6637.0,7093.0,7476.0,9951.0,8060.0,8253.0,9676.0,12438.0,11733.0,15768.0,17901.0,24685.0,19106.0,19869.0,33561.0,19396.0,21641.0,25602.0,26483.0,25199.0,32287.0,32234.0,37985.0,45379.0,53673.0,34110.0,39521.0,47393.0,56973.0,50621.0,66487.0,63545.0,64827.0,76482.0,66095.0,61934.0,57213.0,72412.0
3,Africa,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,3100.0,2800.0,2700.0,2900.0,2500.0,2500.0,2300.0,2600.0,2800.0,3000.0,2500.0,2200.0,3200.0,4300.0,3200.0,3800.0,3200.0,3000.0,4300.0,4600.0,4900.0,4800.0,7800.0,8000.0,8000.0,8700.0,8000.0,10200.0,10740.0,10960.0,11140.0,12020.0,11800.0,11930.0,12190.0,12260.0,12870.0,13020.0,13480.0,14040.0,15240.0,18080.0,19861.0,24279.0,28658.0,31904.0,33206.0,36616.0,52419.0,51080.0,48828.0,48064.0,40418.0,41011.0,46070.0,44620.0,39034.0,32989.0
4,Africa,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,45161.0,46429.0,46078.0,45348.0,44832.0,45456.0,45609.0,45681.0,46071.0,45020.0,45920.0,45736.0,45525.0,50174.0,45700.0,50471.0,48678.0,45520.0,50740.0,50839.0,49951.0,49825.0,49810.0,53000.0,50874.0,50216.0,49975.0,50387.0,36258.0,43638.0,55641.0,52753.0,41777.0,53764.0,60598.0,53446.0,58574.0,67065.0,71600.0,74715.0,79421.0,74968.0,83265.0,82257.0,98835.0,59557.0,78590.0,73047.0,72233.0,79112.0,81414.0,94860.0,114507.0,110086.0,108716.0,110766.0,124970.0,169399.0


In [16]:
crops_all.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38146 entries, 0 to 1653
Data columns (total 67 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Continent     38146 non-null  object 
 1   Area Code     38146 non-null  int64  
 2   Area          38146 non-null  object 
 3   Item Code     38146 non-null  int64  
 4   Item          38146 non-null  object 
 5   Element Code  38146 non-null  int64  
 6   Element       38146 non-null  object 
 7   Unit          38146 non-null  object 
 8   Y1961         22639 non-null  float64
 9   Y1962         22658 non-null  float64
 10  Y1963         22656 non-null  float64
 11  Y1964         22683 non-null  float64
 12  Y1965         22683 non-null  float64
 13  Y1966         22780 non-null  float64
 14  Y1967         22815 non-null  float64
 15  Y1968         22878 non-null  float64
 16  Y1969         22911 non-null  float64
 17  Y1970         22997 non-null  float64
 18  Y1971         23070 non-nul

In [17]:
crops_all.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Continent,38146,5,Europe,10557,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area Code,38146.0,NaN,NaN,NaN,130.297069,75.053504,1.0,63.0,126.0,196.0,299.0
Area,38146,210,"China, mainland",398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item Code,38146.0,NaN,NaN,NaN,614.361663,547.910943,15.0,236.0,446.0,656.0,1841.0
Item,38146,175,"Roots and Tubers, Total",618,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Y2015,33010.0,NaN,NaN,NaN,711842.130233,9973667.916526,0.0,3200.0,23082.5,122509.25,750290277.0
Y2016,33042.0,NaN,NaN,NaN,722969.65786,10249828.135251,0.0,3191.25,22985.0,123213.75,768594154.0
Y2017,33100.0,NaN,NaN,NaN,738894.115952,10171664.733574,0.0,3191.0,22932.0,124309.25,758646205.0
Y2018,33392.0,NaN,NaN,NaN,734436.155127,10245919.307647,0.0,2476.25,21138.0,121818.75,747060316.0


In [18]:
crops_all["Element"].unique()

array(['Area harvested', 'Yield', 'Production'], dtype=object)

It appears that we have data on 3 main elements thanks to this dataset:
- Area harvested
- Yield
- Production

**Let's inspect missing values**

In [19]:
crops_all.isnull().describe().T

,count,unique,top,freq
Continent,38146,1,False,38146
Area Code,38146,1,False,38146
Area,38146,1,False,38146
Item Code,38146,1,False,38146
Item,38146,1,False,38146
...,...,...,...,...
Y2015,38146,2,False,33010
Y2016,38146,2,False,33042
Y2017,38146,2,False,33100
Y2018,38146,2,False,33392


In [20]:
df = pd.DataFrame(crops_all.isnull().sum().sort_index()/len(crops_all))
df.sort_values(by=0,ascending=False).head(50)

,0
Y1961,0.406517
Y1963,0.406071
Y1962,0.406019
Y1965,0.405364
Y1964,0.405364
Y1966,0.402821
Y1967,0.401903
Y1968,0.400252
Y1969,0.399387
Y1970,0.397132


We clearly see that the earlier the data came from, the less information we have, and it's **normal**.

**What is the rank per category & country ?**

In [21]:
continents = crops_all['Continent'].unique()
areas = crops_all['Area'].unique()
items = crops_all['Item'].unique()
elements = crops_all['Element'].unique()

In [30]:
def show_data(df, continent, area, item, element, year):
    year = "Y" + str(year)

    # ALL countries, 1 item, ALL element => 3 barplots ('Total Area Harvested, Yield & production of ITEM per country per YEAR with slider')
    if (area == "ALL") & (element == "ALL"):
        for el in df["Element"].unique():
            to_plot = df[
                (df["Continent"] == continent)
                & (df["Element"] == el)
                & (df["Item"] == item)
            ][[year, "Area"]]
            fig = px.bar(
                to_plot,
                x="Area",
                y=year,
                labels={
                    year: "Total "
                    + str(el)
                    + " ("
                    + str(df[(df["Element"] == el)]["Unit"].unique()[0])
                    + ")",
                    "Area": "Countries",
                },
                title="Total "
                + str(el)
                + " of "
                + str(item)
                + " per country in "
                + str(continent)
                + " ("
                + str(year[1:])
                + ")",
                width=1000,
                height=500,
                template="plotly",
            )

            fig.show()

    # ALL countries, 1 item, 1 element => 1 barplot ('ELEMENT of ITEM per country per YEAR with slider')
    elif (area == "ALL") & (element != "ALL"):
        to_plot = df[
            (df["Continent"] == continent)
            & (df["Element"] == element)
            & (df["Item"] == item)
        ][[year, "Area"]]
        fig = px.bar(
            to_plot,
            x="Area",
            y=year,
            labels={
                year: "Total "
                + str(element)
                + " "
                + df[df["Element"] == element]["Unit"].unique()[0],
                "Area": "Countries",
            },
            title="Total "
            + str(element)
            + " of "
            + str(item)
            + " per country in "
            + str(continent)
            + " ("
            + str(year[1:])
            + ")",
            width=1000,
            height=500,
            template="plotly",
        )
        fig.show()

    # 1 country, 1 item, ALL elements => 3 lineplots ('Area harvested', 'Yield','Production') per YEAR (linegrah, no slider)
    elif area != "ALL":
        if element == "ALL":
            for el in df["Element"].unique():
                tmp_df = df[
                    (df["Continent"] == continent)
                    & (df["Element"] == el)
                    & (df["Area"] == area)
                    & (df["Item"] == item)
                ].drop(
                    columns=[
                        "Continent",
                        "Area Code",
                        "Area",
                        "Item Code",
                        "Item",
                        "Element Code",
                        "Element",
                        "Unit",
                    ]
                )
                to_plot = tmp_df.T.set_axis(["Value"], axis=1, inplace=False)
                to_plot.index = to_plot.index.map(lambda x: x[1:])
                to_plot.index = to_plot.index.map(lambda x: "01/01/" + str(x))
                to_plot.index = pd.to_datetime(to_plot.index)
                to_plot.index.names = ["Year"]
                fig = px.line(
                    to_plot,
                    x=to_plot.index,
                    y="Value",
                    labels={
                        "Value": "Total "
                        + str(el)
                        + " of "
                        + str(item)
                        + " ("
                        + str(df[(df["Element"] == el)]["Unit"].unique()[0])
                        + ")"
                    },
                    title="Evolution of the Total "
                    + str(el)
                    + " of "
                    + str(item)
                    + " in "
                    + str(area),
                    width=1000,
                    height=500,
                    template="plotly",
                )
                fig.show()
        else:
            tmp_df = df[
                (df["Continent"] == continent)
                & (df["Element"] == element)
                & (df["Area"] == area)
                & (df["Item"] == item)
            ].drop(
                columns=[
                    "Continent",
                    "Area Code",
                    "Area",
                    "Item Code",
                    "Item",
                    "Element Code",
                    "Element",
                    "Unit",
                ]
            )
            to_plot = tmp_df.T.set_axis(["Value"], axis=1, inplace=False)
            to_plot.index = to_plot.index.map(lambda x: x[1:])
            to_plot.index = to_plot.index.map(lambda x: "01/01/" + str(x))
            to_plot.index = pd.to_datetime(to_plot.index)
            to_plot.index.names = ["Year"]
            fig = px.line(
                to_plot,
                x=to_plot.index,
                y="Value",
                labels={
                    "Value": "Total "
                    + str(element)
                    + " of "
                    + str(item)
                    + " ("
                    + str(df[(df["Element"] == element)]["Unit"].unique()[0])
                    + ")"
                },
                title="Evolution of the Total "
                + str(element)
                + " of "
                + str(item)
                + " in "
                + str(area),
                width=1000,
                height=500,
                template="plotly",
            )
            fig.show()

In [42]:
show_data(crops_all, "Europe", "France", "Wheat", "Yield", "1986")

**Let's now have a look at the scatter matrix, which is a statistic that is used to make estimates of the covariance matrix, for instance of the multivariate normal distribution.**

In [ ]:
#from pandas.plotting import scatter_matrix

#scatter_matrix(df, figsize=(12, 8))
#plt.show()

**What are the most productive types of crops ? Wheat ? Barley ? Something else ? Let's see**

In [ ]:
## Somme des rendements pour l'année 2019 pour tous les pays, pour les 165 variétés

**What could be interesting is to see the share of production of ANY FOOD ELEMENT per country**

In [286]:
def show_descriptive_scatter(df, item):
    #FIRST, since we do not have the same number of data for the different elements, let's select only the items where we have all the 3 infos
    items_to_drop = list(set(df["Item"].unique()) - set(df[(df["Element"] == "Yield")]["Item"].unique()))
    to_plot = df[~df["Item"].isin(items_to_drop)]
    
    #Let's now drop the columns we do not need
    to_plot = to_plot[(to_plot["Item"] == item)].drop(
        columns=[
            "Area Code",
            "Item Code",
            "Item",
            "Element Code",
            "Unit",
        ]
    )
    #let's deflate our df to put all our "Year" columns in a single column
    to_plot = to_plot.melt(
        id_vars=["Continent", "Area","Element"], var_name="Year", value_name="Value"
    )
    
    # Drop countries without all info because not all countries harvest or have this info for THIS particular item
    countries_to_drop = list(set(to_plot["Area"].unique()) - set(to_plot[(to_plot["Element"] == "Yield")]["Area"].unique()))
    to_plot = to_plot[~to_plot["Area"].isin(countries_to_drop)]
    
    #Some small slicing to get the Year
    to_plot["Year"] = to_plot["Year"].apply(lambda x: x[1:])
    
    #Fill NaN values with 0 to be able to plot
    to_plot["Value"] = to_plot["Value"].fillna(0)
    
    #Define our variables for our plot
    Area_harvested = to_plot[to_plot["Element"] == "Area harvested"]["Value"]
    Production = to_plot[to_plot["Element"] == "Production"]["Value"]
    size = to_plot[to_plot["Element"] == "Yield"]["Value"]
    countries = to_plot[to_plot["Element"] == "Production"]["Area"]
    years = to_plot[to_plot["Element"] == "Production"]["Year"]
    continents = to_plot[to_plot["Element"] == "Production"]["Continent"]
    
    #Plot our scatter plot
    fig = px.scatter(
        to_plot,
        x = Area_harvested,
        y = Production,
        animation_frame=years,
        animation_group=countries,
        size = size,
        color=continents,
        hover_name=countries,
        facet_col=continents,
        log_x=True,
        size_max=30,
        range_x=[1, max(x)],
        range_y=[1, max(y)],
    )
    fig.show()

In [287]:
show_descriptive_scatter(crops_all, "Wheat")

In [98]:
def show_interactive_map(df, item, element):
    to_plot = df[(df["Item"] == item) & (df["Element"] == element)].drop(
        columns=[
            "Area Code",
            "Item Code",
            "Item",
            "Element Code",
            "Element",
            "Unit",
        ]
    )
    to_plot = to_plot.melt(
        id_vars=["Continent", "Area"], var_name="Year", value_name="Value"
    )
    to_plot["Year"] = to_plot["Year"].apply(lambda x: x[1:])
    to_plot["Value"] = to_plot["Value"].fillna(0)
    fig = px.scatter_geo(
        to_plot,
        locations="Area",
        locationmode="country names",  # ‘ISO-3’, ‘USA-states’, or ‘country names’
        color="Continent",  # Values from this column or array_like are used to assign color to marks
        hover_name="Area",
        size="Value",  # size of circles
        animation_frame="Year",
        # projection="equirectangular",
        size_max=50,
        width=1000,
        height=580,
        template="plotly",
        title=f"Evolution of the total {element} of {item}",
    )
    fig.show()

In [101]:
show_interactive_map(crops_all,"Wheat","Production")

## 2. Exploring the Gas Emissions data

**Share of worldwide CO2 emissions per category**

In [ ]:
categories = 

**Let's try to consolidate the CO2 total emissions per year in a new df**

In [ ]:
### MEMO: calculer dans un nouveau df le total d'émissions de CO2 par année

**What are the countries that emit most CO2 emissions ? Is is correlated to their total agricultural production ?**

In [ ]:
## 1. Merge both datasets (total of production, total of emissions) to have a emissions_crops df
## emissions_crops = pd.merge(total_crops_prod,total_emissions, on='',how='inner', suffixes=('_prod','_co2'))
## 2. Plot the corr matrix
## 3. Plot two bar plots: top 10 countries who emit the most // top 10 countries who produce the most

# Machine Learning

## Regression Model

### Data Preprocessing

We choose to preprocess the data according to the use we will make of it. Thus we will preprocess the data in order to have **numerical, approximately symmetrically distributed data** for our **regression** model.

**Some outliers ? Let's make a non-linear re-expression of the dependent variables using the logarithm.**

In [ ]:
# Use of the log function

#We first keep the values inside a list
#list_to_build = df['column'].tolist()

#We then apply the log function on it
#df['column'] = np.log(df['column'])

**Some categorical variables ? Let's encode them into numerical ones to be able to use them in our regression model, and also fill the missing values with the median for each corresponding feature**

In [ ]:
#df['column'].astype('category').cat.codes
#df, y, nas = proc_df(df_raw, 'target_variable') => Target Variable needs to be the CO2 emissions here !!!

**Save the pre-processed data into a *feather* file**

In [ ]:
path_to_tmp = os.path.join(path_to_repo, "data", "tmp")

In [ ]:
path_to_tmp

In [ ]:
os.makedirs(path_to_tmp, exist_ok = True)

In [ ]:
path_to_bulldozers_raw = os.path.join(path_to_tmp, 'bulldozers-raw')

In [ ]:
df_raw.to_feather(path_to_bulldozers_raw)

### Model Building: predict the CO2 emissions of a country depending on its production

**Cross-validation**

**Let's plot the graph**

In [ ]:
#fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (15, 10), dpi = 300)
#plot_tree(model, filled = True)

## Time-series Analysis #1: Worldwide Wheat Production

### Data Preprocessing

In order to be able to make any use of our data in a time series analysis purpose, we will transform our datasets so that we have **dates** as index.

In [ ]:
df = crops_all.set_index(list(crops_all.columns[:7]))
#df = pd.MultiIndex.from_frame(df)
#df = df.to_frame()
df.T.loc[['Y1986']]

In [ ]:
### MEMO: trop complexe, focus simplement sur le blé + enlever les codes

### Analysis

**Resampling & Aggregation**

*Resample data by year using agg() function*

In [ ]:
aggregated = df['Wheat'].resample('Y').agg(['mean','std','min','max'])
aggregated['mean'].plot(label='Mean per Year')
plt.fill_between(aggregated.index,aggregated['max'], aggregated['min'],alpha=0.2,label='min-max per year')
plt.legend()
plt.show()

*Resample & get standard deviation per year to see how much wheat production is volatile*

In [ ]:
#df.loc['1961:2019','Wheat'].resample('Y').std().plot()
#plt.show()

*Resample & plot multiple graphs*

In [ ]:
plt.figure(figsize=(13,9))
df.loc['1961:2019','Wheat'].plot()
df.loc['1961:2019','Wheat'].resample('Y').mean().plot(label="Mean per Year",lw=3,ls=':',alpha=0.8)
df.loc['1961:2019','Wheat'].resample('Decade ?????').mean().plot(label="Mean per Decade",lw=2,ls='--',alpha=0.8)
plt.legend()
plt.show()

**What do we seek to analyse?**<br/>
Before applying any statistical model on a Time Series & forecast some predictions over it, our series has to be on a *stationary state*, which means that, over different time periods,
- the mean should be a *constant* (visually parallel to the x-axis)
- the standard deviation should be a *constant* (visually parallel to the x-axis)
- auto-covariance should not depend on time.

**Why does a time-series has to be stationary to be forecasted?**<br/>
In case a time series has a particular behavior over a time interval, then there's a high probability that over a different interval, this behavior will be the same if the series is **stationary**. Thus it is very helpful to accurately forecast.

**What elements infer on this stationarity?**
Several elements play a role:
- Trend: general direction of a time series over a certain period of time
- Seasonality: seasonal variances (often explained by external factors)
- Noise: irregularity in the data represented by spikes & downs at random
- Cyclicity: cyclic behavior of data

**Moving Average**

In order to first analyse the mean, we could plot the overall average by taking each value & divide by *N*.<br/>But instead of averaging all the values at once, we will compute the mean of the values on a certain ***window*** that we will roll through our sample.<br/> By modifying the **window** parameter, we smooth more or less our mean since we add more or less values as the numerator, over the denominator which is the chosen number of periods to evaluate. <br/><br/>
This allows to erase **transitory fluctuations** & keep the **long-term trend**, but it also gives us a **visual hint** that our time series **is** or **is not** ***stationary***. 

<img src="https://raw.githubusercontent.com/nicoboou/ml_eml/main/img/moving-averages.gif" width="500" align="center">

The successive calculation of moving averages for the same sequence of numbers requires that all the values used by the previous averages be retained ${\bar {x}}_{n}$, in order to replace the oldest term $x_{n-\mathrm {N}}$ with the most recent term $x_{n}$.

$${\displaystyle {\bar {x}}_{n}={\frac {1}{\mathrm {N} }}\ \displaystyle {\sum _{k=0}^{N-1}\;{x_{n-k}}}{\text{, or also }}{\bar {x}}_{n}={\bar {x}}_{n-1}+{\frac {x_{n}-x_{n-\mathrm {N} }}{\mathrm {N} }}}$$

In [ ]:
df.loc['1961:2019','Wheat'].plot()
df.loc['1961:2019','Wheat'].rolling(window=1, freq='Y').mean().plot(label='moving average',lw=3,ls=':',alpha=0.8)
df.loc['1961:2019','Wheat'].rolling(window=1, freq='Y',center=True).mean().plot(label='moving average',lw=3,ls=':',alpha=0.8)

**Exponential Weighted Moving Average (EWMA)**

Compared to simple moving averages, EMAs give greater weight to recent (more relevant) data. <br/>
Indeed in a standard moving average, the oldest price in a fixed series is dropped. By contrast, all prices in a chart influence an exponential moving average: older prices gradually diminish in significance, this time **exponentially** (in contrast with Simple WMA).

The EMA for a series ${\displaystyle T}$ may be calculated recursively:

$${\displaystyle S_{t}={\begin{cases}T_{0},&t=0\\\alpha T_{t}+(1-\alpha )\cdot S_{t-1},&t>0\end{cases}}}$$

Where:
- The coefficient ${\displaystyle \alpha }$ represents the degree of weighting decrease, a constant smoothing factor between 0 and 1. A higher ${\displaystyle \alpha }$ discounts older observations faster.
- ${\displaystyle T_{t}}$ is the value at a time period ${\displaystyle t}$.
- ${\displaystyle S_{t}}$ is the value of the EMA at any time period ${\displaystyle t}$.

More concise way to write it:

$$\bar {x}_{t} = \sum_{n=0}^{+\infty} \alpha ({1-{\alpha}})^n {\bar {x}_{t-1}}$$

In [ ]:
f.loc['1961:2019','Wheat'].plot()
df.loc['1961:2019','Wheat'].rolling(window=1, freq='Y').mean().plot(label='simple moving average',lw=3,ls=':',alpha=0.8)
df.loc['1961:2019','Wheat'].rolling(window=1, freq='Y',center=True).mean().plot(label='centered moving average',lw=3,ls=':',alpha=0.8)

#Here is our new EWMA function
df.loc['1961:2019','Wheat'].ewm(alpha=0.7).mean().plot(label='exponential weighted moving average',lw=3,ls='--')
plt.legend()
plt.show()

Let's compare how the function behaves for different values of $\alpha$

In [ ]:
for x in np.arange(0.1,1,0.1):
    df.loc['1961:2019','Wheat'].ewm(alpha=i).mean().plot(label=f'ewma {i}',lw=3,ls='--')
plt.legend()
plt.show()

***FIRST CONCLUSION: it appears that our time series IS NOT stationary since our rolling mean & rolling std aren't contstant over time***. We will confirm this hypothesis by using the *Augmented Dickey-Fuller statistical test*.

**Augmented Dickey-Fuller Test**<br/>
The *Augmented Dickey–Fuller test* is used in time serie analysis to gives us several values which will help in identifying *stationarity*. We first define the Null hypothesis has that *"a time-series is non-stationary*. Then a  Statistics test & some critical values for some confidence levels are defined & performed. If the test is less than the certain critical values, we can reject the null hypothesis & say that the series is stationary. THE ADCF test also gives us a *p-value*. Acc to the null hypothesis, lower values of p is better.

In [ ]:
#Augmented Dickey–Fuller test:
print('Results of Augmented Dickey Fuller test:')
dick_full_test = adfuller(indexedDataset['#Passengers'], autolag='AIC')

dick_full_output = pd.Series(dick_full_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dick_full_test[4].items():
    dick_full_output['Critical Value (%s)'%key] = value
    
print(dick_full_output)

To be stationary: we should have got:
- the *p-value* as low as possible (0<p-value<1)
- the *critical values* at the different confidence intervals should be as close as possible to the Test Statistic

CONCLUSION: we can see that our time-series is **NOT stationary**. Thus we need to perform some *data transformation* to it.

### Data Transformation

Goal: reach a ***stationary time-series***

**LogScale our data**

We first ScaleTransform our data points using the log function to flatten our spikes.

In [ ]:
#df_logscaled = np.log(df)
#plt.plot(df_logscaled)

**Time Shift Transform**

In [ ]:
#df_logscaled_shifted = df_logscaled - df_logscaled.shift()
#plt.plot(df_logscaled_shifted)

In [ ]:
def stationarity_test(df,column):
    
    #Determine rolling statistics
    movingAverage = df[column].rolling(window=12).mean()
    movingSTD = df[column].rolling(window=12).std()
    
    #Plot rolling statistics
    orig = plt.plot(df[column], color='blue', label='Original')
    mean = plt.plot(movingAverage, color='red', label='Rolling Mean')
    std = plt.plot(movingSTD, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Augmented Dickey–Fuller test:
    print('Results of Augmented Dickey Fuller test:')
    dick_full_test = adfuller(df[column], autolag='AIC')
    dick_full_output = pd.Series(dick_full_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dick_full_test[4].items():
        dick_full_output['Critical Value (%s)'%key] = value

    print(dick_full_output)

In [ ]:
stationarity_test(df_logscaled_shifted,'#Passengers')

### Predictions/Forecasting

**1. State-of-the-art statistical technic: ARIMA**

ARIMA(Auto Regressive Integrated Moving Average) is a combination of 2 models AR(Auto Regressive) & MA(Moving Average). It has 3 hyperparameters - P(auto regressive lags),d(order of differentiation),Q(moving avg.) which respectively comes from the AR, I & MA components. The AR part is correlation between prev & current time periods. To smooth out the noise, the MA part is used. The I part binds together the AR & MA parts.

**How to find the best values P & Q ?**

We need to take help of ACF(Auto Correlation Function) & PACF(Partial Auto Correlation Function) plots. ACF & PACF graphs are used to find value of P & Q for ARIMA. We need to check, for which value in x-axis, graph line drops to 0 in y-axis for 1st time.
- From PACF(at y=0), get P
- From ACF(at y=0), get Q

**Let's plot ACF & PACF functions**

In [ ]:
#ACF & PACF plots

lag_acf = acf(datasetLogDiffShifting, nlags=20)
lag_pacf = pacf(datasetLogDiffShifting, nlags=20, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')            

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
            
plt.tight_layout()

**AR Model**

In [ ]:
#AR Model
#making order=(2,1,0) gives RSS=1.5023
model = ARIMA(indexedDataset_logScale, order=(2,1,0))
results_AR = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues - datasetLogDiffShifting['#Passengers'])**2))
print('Plotting AR model')

**MA Model**

In [ ]:
#MA Model
model = ARIMA(indexedDataset_logScale, order=(0,1,2))
results_MA = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting['#Passengers'])**2))
print('Plotting MA model')

**ARIMA: AR & MA Models Integrated**

In [ ]:
# AR+I+MA = ARIMA model
model = ARIMA(indexedDataset_logScale, order=(2,1,2))
results_ARIMA = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_ARIMA.fittedvalues - datasetLogDiffShifting['#Passengers'])**2))
print('Plotting ARIMA model')

**2. LSTM Model**

In [ ]:
## EXPLAIN HOW A LSTM NETWORK FUNCTIONS STEP BY STEP

## Time Series Analysis #2: Correlation with Pop Growth Forecast 

1. En fonction de l'augmentation de la population, plotter la courbe d'augmentation de la pop -vs- augmentation de la production, et trouver le point de croisement entre les deux

**Let's consolidate the data to get only the total worldwide population per year since 1950**

In [ ]:
world_pop = pop_df_raw[pop_df_raw['Location'] == 'World']
world_pop

**In order to manipulate dates in a simpler manner, let's first tweak our 'Time' column in order to be able to put it in a DateTimeIndex format**

In [ ]:
world_pop['Time'] = world_pop['Time'].apply(lambda x: "01/01/" + str(x))
world_pop = world_pop.set_index('Time')
world_pop.index = pd.to_datetime(world_pop.index)
world_pop

In [ ]:
#Divide estimations & constant/historical data
world_pop_estimations = world_pop['2020':]
world_pop_const = world_pop[:'2019']
world_pop_const

In [ ]:
plt.figure(figsize=(18,10))
world_pop_const['PopTotal'].plot()
plt.show()

In [ ]:
stationarity_test(world_pop_const,'PopTotal')

## Time Series Analysis #3: CO2 Emissions Trend

**Using RandomForests**

1. Analyse des *feature_importances_* d'une regression linéraire sur les prédictions établies par la FAO sur les émissions de CO2

2. Utilisation de ces informations pour construire une time-series analysis et prédire une trend à horizon 2050

3. Comparer ces résultats avec les estimations de la FAO

# Conclusion

Of course these predictions are to be put into perspective and context. Climate change, and more recently Ukrainian war dismantle such algorithms. Such hazards, natural and man-made disasters lead to huge changes in food production. And it is undermining our ability to predict crop yields. Even the best models break down in this context.